Reddit user's sex classificator 


Populating the interactive namespace from numpy and matplotlib


In [220]:
%pylab inline
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package stopwords to /home/ale/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [221]:
train_data = pd.read_csv("/home/ale/Scrivania/reddit_challenge/train_data.csv", encoding="utf8")
clean_train_data=train_data

clean_train_data['word_length'] = clean_train_data['body'].map(lambda x: len(x.strip().split(' ')))

In [222]:
from nltk.stem import LancasterStemmer
stemmer=LancasterStemmer()
#removing double+ spaces
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub('\s+', ' ', x))
clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub(r"http\S+", "", x))

clean_train_data['body']=clean_train_data['body'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


clean_train_data['last_letter'] = clean_train_data['author'].map(lambda x: re.sub(r"\s+$", "", x))


clean_train_data['last_letter'] = clean_train_data['author'].map(lambda x: x[-1])


In [223]:
clean_train_data=clean_train_data.drop(['created_utc'], axis=1)



In [224]:
clean_train_data.head()

,author,subreddit,body
0,Shamus_Aran,mylittlepony,I dont think wed get nearly as much fanfiction...
1,Riddance,sex,Thanks I made it up thats how I got over my fi...
2,Secret_Wizard,DragonsDogma,Are you sure you arent confusing Cyclops the e...
3,Penultimatum,malefashionadvice,dont do this to me bro
4,7-SE7EN-7,todayilearned,Thats what we do when we cant find a mate


In [225]:
clean_train_data.body = clean_train_data.body.str.replace("(<br/>)", "")
clean_train_data.body = clean_train_data.body.str.replace('(<a).*(>).*(</a>)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&amp)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&gt)', '')
clean_train_data.body = clean_train_data.body.str.replace('(&lt)', '')
clean_train_data.body = clean_train_data.body.str.replace('(\xa0)', '')  
#clean_train_data = clean_train_data[clean_train_data['word_length']>=4]
len(clean_train_data)


296042

In [226]:
clean_train_data.head()


,author,subreddit,body
0,Shamus_Aran,mylittlepony,I dont think wed get nearly as much fanfiction...
1,Riddance,sex,Thanks I made it up thats how I got over my fi...
2,Secret_Wizard,DragonsDogma,Are you sure you arent confusing Cyclops the e...
3,Penultimatum,malefashionadvice,dont do this to me bro
4,7-SE7EN-7,todayilearned,Thats what we do when we cant find a mate


In [227]:
clean_train_data.isna().any()

author       False
subreddit    False
body         False
dtype: bool

In [228]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [229]:
clean_train_data['body']=clean_train_data['body'].apply(lambda x: ' '.join([words for words in x.split() if words not in stopwords]))

In [230]:
clean_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296042 entries, 0 to 296041
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   author     296042 non-null  object
 1   subreddit  296042 non-null  object
 2   body       296042 non-null  object
dtypes: object(3)
memory usage: 6.8+ MB


In [231]:
target = pd.read_csv("/home/ale/Scrivania/reddit_challenge/train_target.csv")

In [232]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   author  5000 non-null   object
 1   gender  5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


In [233]:
target.isna().any()

author    False
gender    False
dtype: bool

In [234]:
subreddits = clean_train_data.subreddit.unique()
subreddits_map = pd.Series(index=subreddits, data=arange(subreddits.shape[0]))

In [236]:
from scipy import sparse

In [237]:
def extract_features(group):
    group_subreddits = group['subreddit'].values
    idxs = subreddits_map[group_subreddits].values
    v = sparse.dok_matrix((1, subreddits.shape[0]))
    for idx in idxs:
        if not np.isnan(idx):
            v[0, idx] = 1
    return v.tocsr()

In [238]:
features_dict = {}

for author, group in clean_train_data.groupby('author'):
    features_dict[author] = extract_features(group)

In [239]:
print(features_dict)

{'-Jared': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>, '-Peeter': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>, '-evasian': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>, '-rubiks': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>, '-true_neutral-': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>, '-willis': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>, '00708070': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>, '0200008': <1x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row 

In [240]:
X = sparse.vstack([features_dict[author] for author in target.author])
X

<5000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 49152 stored elements in Compressed Sparse Row format>

In [241]:
y = target.gender

In [242]:
def extract_text(group):
    group_text = group['body'].values
    return " ".join(group_text)

In [243]:
text_dict = {}

for author, group in train_data.groupby('author'):
    text_dict[author] = extract_text(group)

In [244]:
author_text = [text_dict[author] for author in target.author]
author_text[:2]

['I still prefer to buy foods either grown locally or where animals are treated better but this definitely has me looking at organic food differently',
 'A name friend Ten points to Gryffindor Hyperbole and a Half has the best explanation Ive ever seen My friends who struggle with depression and I all agree that its like shes inside our heads If your PC has decent specs definitely get it on Steam and learn your way around Nexus Mod Manager Its not too much trouble to get a few simple graphics and content mods And I strongly recommend using the mouse and keyboard Hotkeys are a must Either really learn how to play guitar because you want to make music or dont bother If youre not invested in cultivating a talent its just a waste of money and anyone who might be attracted to you at first will leave when they find out youre just an attentionseeking fake I think you are confusing the laymans or colloquial meaning of theory with the scientific definition In science a theory is an idea that ha

In [305]:
##Model
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(C=1.2, max_iter=1000)
clf_lr.fit(X, y)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)

In [306]:
clf_lr.fit(X_train, y_train)

LogisticRegression(C=2.3, max_iter=1000)

In [307]:
y_pred_lr = clf_lr.predict(X_test)

In [308]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_lr))

0.876


In [296]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred_lr))

              precision    recall  f1-score   support

           0       0.86      0.96      0.91       373
           1       0.82      0.53      0.64       127

    accuracy                           0.85       500
   macro avg       0.84      0.74      0.77       500
weighted avg       0.85      0.85      0.84       500



In [310]:
from sklearn.metrics import confusion_matrix 

In [311]:
confusion_matrix(y_true=y_test, y_pred=y_pred_lr)

array([[360,  14],
       [ 48,  78]])

In [312]:
from sklearn.model_selection import KFold, cross_val_predict
kfold_lr = KFold(n_splits=5)
result_lr = cross_val_predict(estimator=clf_lr,X=X,y=y,cv=kfold_lr)

In [313]:
from sklearn.metrics import f1_score
f1_lr=f1_score(y, result_lr)

In [314]:
f1_lr

0.6761658031088082

In [316]:
from sklearn import metrics 
acc_lr = metrics.accuracy_score(y, result_lr)

In [317]:
acc_lr

0.85

In [318]:
from sklearn.metrics import roc_auc_score

In [319]:
roc_auc_score_lr = roc_auc_score(y, y_score=result_lr)

In [320]:
roc_auc_score_lr 

0.7650163983221794

Support vector machine

In [321]:
##Model
from sklearn.svm import SVC
clf_svc = SVC(kernel='linear', probability=True)
clf_svc.fit(X, y)

SVC(kernel='linear', probability=True)

In [332]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [333]:
clf_svc.fit(X_train, y_train)

SVC(kernel='linear', probability=True)

In [334]:
y_pred_svc = clf_svc.predict(X_test)

In [335]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_svc))

0.834


In [336]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred_svc))

              precision    recall  f1-score   support

           0       0.87      0.91      0.89       720
           1       0.73      0.64      0.68       280

    accuracy                           0.83      1000
   macro avg       0.80      0.78      0.79      1000
weighted avg       0.83      0.83      0.83      1000



In [337]:
confusion_matrix(y_true=y_test, y_pred=y_pred_svc)

array([[654,  66],
       [100, 180]])

In [338]:
kfold_svc = KFold(n_splits=5)
result_svc = cross_val_predict(estimator=clf_svc,X=X,y=y,cv=kfold_svc)

In [339]:
f1_svc=f1_score(y, result_svc)

In [340]:
f1_svc

0.6649874055415618

In [341]:
acc_svc = metrics.accuracy_score(y, result_svc)

In [342]:
acc_svc

0.8404

In [343]:
from sklearn.metrics import roc_auc_score

In [344]:
roc_auc_score_svc = roc_auc_score(y, y_score=result_svc)

In [345]:
roc_auc_score_svc

0.7605461220957772

In [348]:
from sklearn.model_selection import cross_val_score
cross_val_svc = cross_val_score(SVC(kernel='linear', probability=True),X,y, cv=5, scoring='accuracy')
print (cross_val_svc)
print (cross_val_svc.mean())

[0.828 0.858 0.828 0.845 0.843]
0.8404


Random Forest



In [352]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
clf_1 = RandomForestClassifier()
Cs_1 = [10, 500]
Cs_2 = [None, 50, 400]
grid_1 = GridSearchCV(clf_1, param_grid={'n_estimators': Cs_1,'max_depth': Cs_2}, cv=5)
grid_1.fit(X, y)
print ("best parameter choice:", grid_1.best_params_)

best parameter choice: {'max_depth': 50, 'n_estimators': 500}


In [356]:
print(grid_1.best_params_['n_estimators'])

500


In [357]:
clf_rf = RandomForestClassifier(n_estimators=grid_1.best_params_['n_estimators'], max_depth=grid_1.best_params_['max_depth'])

In [366]:
X_train, X_test, y_train, y_test, = train_test_split(X,y, test_size=0.3)

In [367]:
clf_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=50, n_estimators=500)

In [368]:
y_pred_rf = clf_rf.predict(X_test)

In [369]:
print(accuracy_score(y_test, y_pred_rf))

0.8373333333333334


In [370]:
print(classification_report(y_true=y_test, y_pred=y_pred_rf))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90      1108
           1       0.90      0.43      0.58       392

    accuracy                           0.84      1500
   macro avg       0.86      0.70      0.74      1500
weighted avg       0.85      0.84      0.82      1500



In [371]:
confusion_matrix(y_true=y_test, y_pred=y_pred_rf)

array([[1089,   19],
       [ 225,  167]])

In [372]:
kfold_rf = KFold(n_splits=5, random_state=42)
result_rf = cross_val_predict(estimator=clf_rf,X=X,y=y,cv=kfold_rf) 

In [373]:
f1_rf=f1_score(y, result_rf)
f1_rf

0.5975186104218362

In [375]:
acc_rf = metrics.accuracy_score(y, result_rf)
acc_rf

0.8378

In [376]:
roc_auc_score_rf = roc_auc_score(y, y_score=result_rf)
roc_auc_score_rf 

0.7143635211490947

In [377]:
classifiers = ['Logistic Regression', 'Random Forest Classifier', 'SVC']
acc = np.array([acc_lr, acc_rf, acc_svc])
max_acc = np.argmax(acc)
rocauc = np.array([roc_auc_score_lr, roc_auc_score_rf, roc_auc_score_svc])
max_rocauc = np.argmax(rocauc)
print (classifiers[max_acc] + ' is the best classifier for this problem')

Logistic Regression is the best classifier for this problem


In [383]:
MultiNB=MultinomialNB()

In [387]:
MultiNB.fit(X_train,y_train)

MultinomialNB()

In [388]:
y_pred_MultiNB = MultiNB.predict(X_test)



In [390]:
print(accuracy_score(y_test, y_pred_MultiNB))

0.8566666666666667
